# Exercise Sheet No. 4

---

> Machine Learning for Natural Sciences, Summer 2021, Jun.-Prof. Pascal Friederich, pascal.friederich@kit.edu
> 
> Deadline: 10.05.2021, 8am

---

**Topic**: This exercise sheet will focus on the math basics for ML.

**[Before you start, please fill out our survey for the exam!](https://forms.gle/poPp6yY8TxHcPv3E7)**

We will use seaborn as a more abstract plotting interface in this exercise. You can install it into your current conda environment by executing the following code cell:

In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} seaborn

# Linear Regression
In assignment 2, we have seen the use of decision tree for iris species classification. Now, let's try to solve the same problem with another simple machine learning technique: linear regression (LR).   

Linear regression uses a linear combination of features to predict a target. In our case we can use a linear combination of the four flower descriptors to predict the species.

In this assignment, you will learn to implement the Loss function and Gradient Decent for optimization. Then, we will see that even small model like LR can overfit, and how regularization (ridge regression) can help against this problem.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import random
from functools import wraps
warnings.filterwarnings('ignore')

## Preprocessing
Let's start with preprocessing the dataset, as already learned from assignment 2.

In [ ]:
def log_name(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f"\n{func.__name__}:")
        return result
    return wrapper

def log_shape(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f"\tshape: {result.shape}")
        return result
    return wrapper

def log_columns(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f"\tcolumns: {result.columns.values}")
        return result
    return wrapper

@log_columns
@log_shape
@log_name
def load(df, path):
    """Loads the dataset from path."""
    df = pd.read_csv(path)
    return df

@log_columns
@log_shape
@log_name
def convert_to_categorical(df, col_name: str):
    df[col_name] = df[col_name].astype('category')
    return df

@log_columns
@log_shape
@log_name
def add_class_labels(df):
    df['class'] = df['species'].cat.codes
    return df

In [ ]:
df = pd.DataFrame()
df = (
    df.pipe(load, 'iris.csv')
      .pipe(convert_to_categorical, 'species')
      .pipe(add_class_labels)
)

## Plot the data
It is always a good practice to have some inspection on the dataset. This provides us with information about the data such as its structure, range, outliers etc., which may help on the design of ML algorithm. There are numerous ways to visualize data information, and introduced here is the one called "Pairplot", which displays pairwise relationships in a dataset. Pairplot can be implemented easily with the [seaborn](https://seaborn.pydata.org/generated/seaborn.pairplot.html?highlight=pairplot#seaborn.pairplot) library. Here we used it to show the pairwise relationships among iris features.

In [ ]:
sns.pairplot(df.loc[:,:'species'], hue= 'species')
plt.show()

Question: out of these combinations of features, which pair has the strongest linear correlation relationship?

**1.** sepal_length and petal_width    
**2.** sepal_width and petal_length    
**3.** petal_length and petal_width    
**4.** sepal_width and petal_length    
**5.** sepal_length and sepal_width

(To learn more about the linear correlation, please refer to [here](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient))

Assign the number of your choice to the variable `A`:

In [ ]:
A = (
# YOUR CODE HERE
raise NotImplementedError()
)

## Generate training/validation set
Now, for our machine learning task, let's extract feature matrix `X` and label matrix `Y` from the dataframe `df`. Please implement the `split_X_Y` function that returns feature `X` as a $150 \times 4$ numpy array, as well as label `Y` as a $150 \times 1$ numpy array. `X` should contain values from column "sepal_length", "sepal_width", "petal_length" and "petal_width". `Y` should be the value of column "class". You may use methods such as `.reshape()` etc., and attributes like `.values`.

In [ ]:
def split_X_Y(df):
    """split the dataframe into feature matrix X and label matrix Y"""
    # YOUR CODE HERE
    raise NotImplementedError()
    return X, Y

In [ ]:
X, Y = split_X_Y(df)
assert X.shape == (150, 4)
assert Y.shape == (150, 1)

We have learned that the linear model tries to fit the function:
\begin{align}
y = \omega^T x + \omega_0 = \sum^n_{i=1} \omega_i x_i + \omega_0
\end{align}
Where $\omega$ is the weight vector and $\omega_0$ is the bias term. Let $x_0 = 1$, this function can be rewritten into:
\begin{align}
y =  \sum^n_{i=1} \omega_i x_i + \omega_0 x_0 = \sum^n_{i=0} \omega_i x_i = X \Omega^T
\end{align}
$\Omega$ can be initialized randomly. For our case it has five element - the first for the bias and four for the features.

In [ ]:
np.random.seed(0)
omega = np.random.randn(1,5)
omega

To contract the bias term and the features to obtain $X$, we can stack a column to feature matrix `X` with all values equal to $1$. This may be easily implemented with the numpy method `.hstack()` . You may find more details [here](https://numpy.org/doc/stable/reference/generated/numpy.hstack.html).

In [ ]:
X = (
# YOUR CODE HERE
raise NotImplementedError()
)

In [ ]:
assert X.shape == (150, 5)

There's a "closed-form solution" that estimates the best parameter set $\Omega^*$ by solving the equation:
\begin{align}
\Omega^* = (X^TX)^{-1}X^Ty
\end{align}
Watch this [video](https://www.coursera.org/lecture/ml-regression/approach-1-closed-form-solution-G9oBu) from coursera for detailed explanation.

## Gradient Decent
For more complex tasks a closed form solution often doesn't exist and hence we will introduce another approach, aka Gradient Decent (GD) algorithm, to solve the linear regression. Gradient Decent works by updating the weight vector incrementally after each epoch. Read more [here](https://developers.google.com/machine-learning/crash-course/reducing-loss/gradient-descent).

Firstly, let's do a train-valid split for our dataset, as we already saw in Assignment 2. The `x_train` and `y_train` will be our training set, and `x_val`, `y_val` will be our validation set.

There's also a useful method `sklearn.model_selection.train_test_split()` from scikit-learn for the same task. You can find its documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

In [ ]:
idx = list(range(0, 150))
random.shuffle(idx)
x_train = X[idx[:120]]
y_train = Y[idx[:120]]
x_val = X[idx[120:]]
y_val = Y[idx[120:]]

### Cost function
The mean squared error is a common loss function that measures the average squared difference between predict and real values. Here, it is defined as:
\begin{align}
MSE = \frac{1}{2n}\sum^n_{i=1}(\text{y_pred}_i - \text{y_real}_i)^2
\end{align}
The $\frac{1}{2}$ in equation is just for convenience when computing the gradient (see next step). Recall that $y = X \Omega^T$, so we have:
\begin{align}
MSE = \frac{1}{2n}\sum^n_{i=1}(x_i \Omega^T - \text{y_real}_i)^2
\end{align}
Please implement the `mean_square_error()` that returns MSE. Methods/attribute you may need are `numpy.sum()`, `numpy.dot()` and `ndarray.T`. Although we use a $\sum$ in the notation you can use your knowledge from exercise 3 to compute the following using only numpy vectors and matrices.

In [ ]:
def mean_square_error(x, y, omega):
    """
    return the mean suqared error.
    
    Args:
        x: numpy array of features.
        y: numpy array of corresponding classes.
        omega: weight vector.
        
    Returns:
        mse: mean squared error.
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return mse

In [ ]:
a = np.array([[1.0, 2.0], [2.0, 2.0]])
b = np.array([[1.0], [2.0]])
w = np.array([[1.0, 1.0]])
assert mean_square_error(a, b, w) == 2.0

After each epoch $t$, the Gradient Decent algorithm updates the weight vector in the direction of the negative gradient in order to reduce the cost function. The gradient is simply the partial derivative of mean squared error to the weight.

You can deduct the derivative yourself for practice. Just run the following cell to render the answer:

In [ ]:
from IPython.display import display, Markdown

display(Markdown("\\begin{align}"
                 "\\frac{\\partial MSE}{\\partial \\Omega} &= "
                 "\\frac{1}{n} \\sum^n_{i=0}(x_i \\Omega^T - \\text{y_real}_i) x_i \\\\"
                 "\\Omega(t+1) &= \\Omega(t) - \\alpha \\frac{\\partial MSE}{\\partial \\Omega(t)}"
                 "= \\Omega(t) - \\frac{\\alpha}{n} \\sum^n_{i=0}(x_i \\Omega^T - \\text{y_real}_i) x_i"
                 "\\end{align}"))

The $\alpha$ is the learning rate that controls the step size for the update after each iteration.

Please implement the `weight_update_function()` that returns updated $\Omega$. Methods/attribute you may need are `numpy.dot()`, `numpy.reshape()` and `ndarray.T`.

In [ ]:
def weight_update_function(x, y, omega, alpha):
    """
    return updated set of weights
    
    Args:
        x: numpy array of features.
        y: numpy array of corresponding classes.
        omega: weight vector.
        alpha: learning rate.
        
    Returns:
        omega_updated: the updated weight vector.
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return omega_updated

In [ ]:
assert abs(np.mean(weight_update_function(a, b, w, 0.001)) - 0.996) <= 0.001

To monitor the training process, here we use mean absolute error:
\begin{align}
MAE = \frac{1}{n}\sum^n_{i=1} |\text{y_pred}_i - \text{y_real}_i|
\end{align}
Please implement the `mean_absolute_error()` that returns MAE. Methods/attribute you may need are `numpy.sum()`, `numpy.dot()`, `numpy.abs()` and `ndarray.T`.

In [ ]:
def mean_absolute_error(x, y, omega):
    """
    return the mean absolute error
    
    Args:
        x: numpy array of features.
        y: numpy array of corresponding classes.
        omega: weight vector.
        
    Returns:
        mse: mean absolute error.
        """
    # YOUR CODE HERE
    raise NotImplementedError()
    return mae

In [ ]:
assert np.round(mean_absolute_error(a, b, w))  == 2.0

For the last step before training the model, let's set up the learning rate and number of iterations. We use `J_train` and `J_val` to record mean absolute errors of the training and validation processes.

In [ ]:
epochs = 5000 # number of updates to the weight
alpha = 0.01 # learning rate
J_train = np.zeros(epochs) # MAE of training process
J_val = np.zeros(epochs) # MAE of validation process

### Training & plot
Now, let's train our model with the `weight_update_function()` and record mean absolute errors through `mean_absolute_error()` for the training/validation process.

In [ ]:
np.random.seed(0)
omega = np.random.randn(1,5)
for i in range(epochs):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert J_train[-1] > 0

Here the `plot_training_curve()` is implemented to visualize the training process.

In [ ]:
def plot_training_curve(MAE_train, MAE_val, epochs):
    """Plot the mean absolute error for training/validation process"""
    fig, ax = plt.subplots(figsize=(7.5, 5))
    ax.plot(np.arange(epochs), MAE_train, label='Training')
    ax.plot(np.arange(epochs), MAE_val, label='Validation')
    ax.set_ylim([0,1])
    ax.set_ylabel("Mean Absolute Error")
    ax.set_xlabel("Epochs")
    ax.set_title("Mean Absolute Error vs Epochs")
    ax.legend(loc='upper right')
    plt.show()

In [ ]:
plot_training_curve(J_train, J_val, epochs)

### Accuracy
Now, with the optimized weight vector, let's implement `prediction()` to get the predicted class labels from our linear model. You may need `numpy.dot()` and `ndarray.T` for the computation, and `numpy.round()` to round up the results into integers (because our class labels are integer 0, 1, 2).

In [ ]:
def prediction(X, omega):
    """
    Return predicted labels.
    
    Args:
        X: the feature matrix.
        omega: optimized weight vector
        
    Returns:
        Y_pred: predicted labels.
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return Y_pred

In [ ]:
Y_pred = prediction(X, omega)
assert Y_pred.shape == Y.shape

With `Y_pred`. Please implement `accuracy()` that calculates the prediction accuracy.

In [ ]:
def accuracy(Y_pred, Y):
    """
    Compute the accuracy of prediction.
    
    Args:
        Y_pred: the predicted class labels.
        Y: the real class labels.
        
    Returns:
        acc: calculated accuracy in float.
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return acc

In [ ]:
acc = accuracy(Y_pred, Y)
acc

What's the accuracy of your model? Please input it below (with 4 significant figures).

In [ ]:
my_accuracy = (
    # YOUR CODE HERE
    raise NotImplementedError()
)

### Visualize predictions vs ground truth
For the last step, let's visualize our predictions v.s. the real labels.

In [ ]:
def plot_predict_vs_real(Y_pred, Y):
    """Plot the predictions vs ground truch"""
    fig, ax = plt.subplots(figsize=(7.5, 5))
    ax.scatter(np.arange(1, 151, 1), Y_pred, label='Predictions')
    ax.plot(np.arange(1, 151, 1), Y, label='Ground Truch', color='red')
    ax.legend(loc="upper left")
    plt.yticks(np.arange(0, 3, 1))
    plt.show()

In [ ]:
plot_predict_vs_real(Y_pred, Y)

We can also write confusion matrix as assignment 2 to visualize the results.

In [ ]:
def confusion_table(Y_pred, Y):
    """Plot the confusion table"""
    confusion_matrix = pd.DataFrame(index=df['species'].unique(),
                                columns=df['species'].unique(),
                                data = np.zeros((3,3)))
    confusion_matrix.index.name = 'True'
    confusion_matrix.columns.name = 'Predicted'
    # YOUR CODE HERE
    raise NotImplementedError()
    print(confusion_matrix)

In [ ]:
confusion_table(Y_pred, Y)

How many points are mis-classified? Please input you answer below:

In [ ]:
mis_classified = (
    # YOUR CODE HERE
    raise NotImplementedError()
)

Please feel free to play around with the model and training process. You may adjust the learning rate and number of epochs to see how the trainig curve changes (do not forget to re-initialize the weight vector). You may also try different train-validation split ratio to see the effect.

## Overfitting & Ridge regression

When finishing this assignment, you may (and may not, depending on the training set setup) sometimes observe the gap between the training curve and validation curve (validation error higher than training error) that cannot be diminished by increasing the number of epochs. This phenomenon is known as "overfitting". It happens when the model gets too complex so that it fits the training set perfectly, but loses the generalization ability towards unseen data from validation/test set.

Ridge regression, also known as L2 regularization, is a useful technique to restrict our model from getting too complicated and reduce the effect of overfitting. It works by simply adding a penalty term to the cost function. In our case, the penalty term is $||\Omega||^2$. It prefers lower absolute values of weights thus reduce the model complexity. For more information, please refer to [here](https://developers.google.com/machine-learning/crash-course/regularization-for-simplicity/l2-regularization).

By introducting the penalty term, our cost function becomes:
\begin{align}
MSE = \frac{1}{2}(\frac{1}{n}\sum^n_{i=1}(\text{y_pred}_i - \text{y_real}_i)^2 + \lambda ||\Omega||^2)
\end{align}
$\lambda$ is the coefficient to adjust the regularization effect.

While the mean absolute error calculation stays the same, the weight update function becomes:
\begin{align}
\Omega = \Omega - \alpha \frac{\partial MSE}{\partial \Omega} = \Omega - \alpha(\frac{1}{n} \sum^n_{i=0}(x_i \Omega^T - \text{y_real}_i) x_i + \lambda ||\Omega||) 
\end{align}
Now, please implement the new cost function `mean_square_error_ridge()` and `weight_update_function_ridge()` with the penalty term.

In [ ]:
def mean_square_error_ridge(x, y, omega, lam):
    """
    return the mean suqared error.
    
    Args:
        x: numpy array of features.
        y: numpy array of corresponding classes.
        omega: weight vector.
        lam: ridge regression coefficient.
        
    Returns:
        mse: mean squared error.
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return mse

In [ ]:
a = np.array([[1.0, 2.0], [2.0, 2.0]])
b = np.array([[1.0], [2.0]])
w = np.array([[1.0, 1.0]])
assert np.round(mean_square_error_ridge(a, b, w, 0.1), 1) == 2.1

In [ ]:
def weight_update_function_ridge(x, y, omega, alpha, lam):
    """
    return updated set of weights
    
    Args:
        x: numpy array of features.
        y: numpy array of corresponding classes.
        omega: weight vector.
        alpha: learning rate.
        lam: ridge regression coefficient.
        
    Returns:
        omega_updated: the updated weight vector.
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return omega_updated

In [ ]:
assert abs(np.mean(weight_update_function_ridge(a, b, w, 0.001, 0.1)) - 0.99) <= 0.01

Now, let's look at a very unbalanced training set. In this example, only 25 instances are used as training set ($17\%$), and the number of class 0 is significantly higher than both class 1 and 2.

In [ ]:
import pickle
with open('iris_overfit', 'rb') as f:
    x_train, x_val, y_train, y_val = pickle.load(f)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(np.arange(0, len(y_train), 1), y_train)
plt.yticks(np.arange(0, 3, 1))
ax.set_xlabel("data index")
ax.set_ylabel("class label")
ax.set_title("Class distribution of training set")
plt.show()

For the training, let's do the normal linear regression and ridge regression side by side, with `J_train`, `J_val` recording linear regression training/validation MAE, and `J_train_ridge`, `J_val_ridge` recording ridge regression MAE. Please implement them in the same `for` loop.

In [ ]:
np.random.seed(0)
omega = np.random.randn(1,5)
epochs = 3000 # number of updates to the weight
alpha = 0.0005 # learning rate
lam = 0.5 # coefficient of L2 penalty
J_train = np.zeros(epochs) # record MAE of training process (without penalty)
J_val = np.zeros(epochs) # record MAE of validation process (without penalty)
J_train_ridge = np.zeros(epochs) # record MAE of training process (with penalty)
J_val_ridge = np.zeros(epochs) # record MAE of validation process (with penalty)

In [ ]:
omega_norm = np.copy(omega) # weight vector for training without penalty
omega_ridge = np.copy(omega) # weight vector for ridge regression

for i in range(epochs):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert J_train_ridge[-1] > 0

Now we plot the training curves again, but this time on a log-log scale to better see the differences:

In [ ]:
def plot_training_curve_ridge(MAE_train, MAE_val, MAE_train_ridge, MAE_val_ridge, epochs):
    """Plot the mean absolute error for training/validation process"""
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    ax[0].loglog(np.arange(epochs), MAE_train, label='Training')
    ax[0].loglog(np.arange(epochs), MAE_val, label='Validation')
    #ax.set_ylim([0,1])
    ax[0].set_ylabel("Mean Absolute Error")
    ax[0].set_xlabel("Epochs")
    ax[0].set_title("Without L2 Regularization")
    ax[0].legend(loc='upper right')
    
    ax[1].loglog(np.arange(epochs), MAE_train_ridge, label='Training')
    ax[1].loglog(np.arange(epochs), MAE_val_ridge, label='Validation')
    #ax.set_ylim([0,1])
    ax[1].set_ylabel("Mean Absolute Error")
    ax[1].set_xlabel("Epochs")
    ax[1].set_title("With L2 Regularization")
    ax[1].legend(loc='upper right')
    plt.show()

In [ ]:
plot_training_curve_ridge(J_train, J_val, J_train_ridge, J_val_ridge, epochs)

In [ ]:
Y_pred = prediction(X, omega_norm)
Y_pred_ridge = prediction(X, omega_ridge)
acc = accuracy(Y_pred, Y)
acc_ridge = accuracy(Y_pred_ridge, Y)
print(acc)
print(acc_ridge)

In [ ]:
plot_predict_vs_real(Y_pred, Y)
plot_predict_vs_real(Y_pred_ridge, Y)

Yes or No question: from the plots/accuracy calculations, do you think the ridge regression improved the learning performance?

In [ ]:
Answer = (
    # YOUR CODE HERE
    raise NotImplementedError()
)

## Scipy Optimize

Rather than handcraft the Gradient Decent algorithm, there are libraries that can do the optimization task automatically. The `minimize()` function provided by SciPy library is a good example. It takes as input the name of the function that needs to be minimized, the initial point where the search starts and (optionally) the name of a specific search algorithm. It returns a `OptimizeResult` object that contains details of the optimization result. Below is an example of using `minimize()` function to optimize our linear model.

In [ ]:
from scipy.optimize import minimize

np.random.seed(0)
omega = np.random.randn(1,5)

def mean_square_error(omega):
    return (1/(2*len(X)) * np.sum((np.dot(X, omega.reshape(1,5).T) - Y) ** 2 ))


res = minimize(mean_square_error, omega, method='L-BFGS-B')
print(res)
omega_op = np.array(res.x).reshape(1,5)

In [ ]:
Y_pred = prediction(X, omega_op)
acc = accuracy(Y_pred, Y)
acc

In [ ]:
plot_predict_vs_real(Y_pred, Y)

You can find a good tutorial of SciPy optimization [here](https://machinelearningmastery.com/function-optimization-with-scipy/).  

If you have more questions, please refer to the SciPy [documentation](https://docs.scipy.org/doc/scipy/reference/optimize.html).

For the introduction of the "L-BFGS-B" search algorithm using here, please read [this](http://sepwww.stanford.edu/data/media/public/docs/sep117/antoine1/paper_html/node6.html).

***Please do this in a separate notebook that is not submitted!***

**[If you didn't do it in the beginning please fill out the survey for the exam now!](https://forms.gle/poPp6yY8TxHcPv3E7)**